## Add and push new simulations to the CoReDB - Notebook

This notebook:
1. Adds simulations to the `dbkeys` and prepares its corresponding folder in the `CoRe_DB_clone` path
2. Adds their runs with metadata and h5 file.
3. Updates the CoRe DB index json file
4. Writes in each simulation folder a `.gitattributes` file to add the `data.h5` file to the LFS storage
5. Initializes and creates the git repositories 
6. Pushes everything to [https://core-gitlfs.tpi.uni-jena.de/](https://core-gitlfs.tpi.uni-jena.de/)

Note: The repositories are created as *private* projects. For now it's only possible to change this on the website under `Settings -> Visibility, project features, permissions` for each repo.

Pre-requisites:
- Make sure you have enough access rights to push
- Add your ssh-key to `gitlab` [here](https://core-gitlfs.tpi.uni-jena.de/-/profile/keys). If it had a passphrase, avoid typing it every time by:
  - Starting the **ssh-agent**: ``` eval `ssh-agent -s` ```
  - Add key and enter passphrase: `ssh-add ~/.ssh/keyname` and you're set!
- This script assumes that the simulations already exist in a common directory (in this case in `tullio:/data/numrel/DATABASE/Release02/`) with their correct run folders with `data.h5` + `metadata.txt`
- In this particular case we had already the `metadata_main.txt` and `metadata.txt`'s, but should work anyways by defining your own metadata dictionary.
- Assumes the CoRe_DB_clone directory is in `tullio:/data/numrel/DATABASE/CoRe_DB_clone`

[*Last Updated: 10/2022 AG*]

In [1]:
import os
import numpy as np
from watpy.coredb.coredb import *
from watpy.coredb.metadata import *
from watpy.utils.ioutils import *
from watpy.utils.coreh5 import *

The function below does the git magic. Modify if you find a way to make the repo public from the beginning.

In [2]:
def git_push(dir,entry,verbose=True):
    print("Working in ",dir)
    out, err = runcmd(['git','init'],dir,True)
    print(out,err)
    out, err = runcmd(['git','add','.'],dir,True)
    print(out,err)
    commit = 'Initial commit'
    out = runcmd(['git','commit','-m',commit], dir, True)
    print(out,err) 
    url = 'git@core-gitlfs.tpi.uni-jena.de:core_database/'+entry+'.git'
    out = runcmd(['git','push','--set-upstream',url,'master'],dir,True)
    print(out,err)
    print("done!")

In [ ]:
db_path = '/data/numrel/DATABASE/CoRe_DB_clone'  
rel2_path   = '/data/numrel/DATABASE/Release02/' # Change accordingly!
cdb = CoRe_db(db_path)
idb = cdb.idb

direcs = os.listdir(rel2_path) # Array with all simulation folders
# Delete the folders not corresponding to simulations:
direcs.remove('Sim')
direcs.remove('README')
direcs.sort() # Sort if necessary

In [ ]:
### ADD SIM DIRECTORY AND TO DBKEYS
for entry in direcs:
    dbkey = entry.replace('_',':')
    code = entry.split('_')[0]
    this_path = rel2_path + entry
    
    sim = CoRe_sim(this_path)
    
    # Prepare metadata (change for a dictionary if necessary)
    metamain = this_path + '/metadata_main.txt'
    meta_main = sim.md.data

    # Add dbkey
    newdbkey = cdb.add_simulation(code,meta_main['simulation_name'], metadata = meta_main)

    # Same for the runs
    run_list = list(sim.run.keys())
    run_list.sort()

    for res in run_list:
        print('In :', res)
        run_path = this_path+'/'+res
        simrun = sim.run[res]
        meta_dict = simrun.md.data # metadata.txt should exist!
        cdb.sim[newdbkey].add_run(path = run_path , metadata = meta_dict)

In [ ]:
### UPDATE CORE INDEX
# read metadata_main in a list
mdlist = []
print(cdb.idb.dbkeys)
for key in idb.dbkeys:
    mdlist.append(cdb.sim[key].md)

# update the index 
idb.update_from_mdlist(mdlist)

# write the index to JSON with the appropriate template 
idb.to_json_tmplk()

Now prepare the git repositories:

In [ ]:
for entry in direcs[3:]:
    print('In: ', entry)
    git_path = db_path + '/' + entry
    sim = CoRe_sim(git_path)
    run_list = list(sim.run.keys())
    run_list.sort()
    
    ## ADD .GITATTRIBUTES FILE
    file_path = git_path + '/.gitattributes'
    if os.path.exists(file_path):
        print('File exists!')
    else:
        git_file = open(file_path, "w")
        L = []
        for res in run_list:
            git_string = res+'/data.h5 filter=lfs diff=lfs merge=lfs -text \n'
            L.append(git_string)
        git_file.writelines(L)
        git_file.close()
        print('Wrote file!')

    ## CREATE REPO AND PUSH
    print('Initialize git repo ..')
    git_push(git_path,entry)